<a href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/boost_massively_imbalanced_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
%%capture
!pip install pyyaml numpy pandas sklearn smart_open xgboost
!pip install -U gretel-client

In [1]:
# Specify your Gretel API key

import pandas as pd
from gretel_client import configure_session

pd.set_option("max_colwidth", None)

configure_session(api_key="prompt", cache="yes", validate=True)


Found cached Gretel credentials
Using endpoint https://api-dev.gretel.cloud
Logged in as drew@gretel.ai ✅


In [2]:
# Create imbalanced train and test data
# We will use sklearn's make_classification to create a test dataset.
# Or, load your own dataset as a Pandas DataFrame.

CLASS_COLUMN = "Class"  # the labeled classification column
CLASS_VALUE = 1  # the minority classification label to boost
MAX_NEIGHBORS = 5  # number of KNN neighbors to use per positive datapoint
SYNTHETIC_PERCENT = 10  # generate SYNTHETIC_PERCENT records vs. source data

# Create imbalanced test dataset
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


n_features = 15
n_recs = 10000


def create_dataset(n_features: int) -> pd.DataFrame:
    """Use sklearn to create a massively imbalanced dataset"""
    X, y = make_classification(
        n_samples=n_recs,
        n_features=n_features,
        n_informative=10,
        n_classes=2,
        weights=[0.95],
        flip_y=0.0,
        random_state=42,
    )

    df = pd.DataFrame(X, columns=[f"feature_{x}" for x in range(n_features)])
    df = df.round(6)
    df[CLASS_COLUMN] = y
    return df


dataset = create_dataset(n_features=n_features)
train, test = train_test_split(dataset, test_size=0.2)

train.head()


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,Class
536,-1.304168,-1.787931,0.726976,-2.607825,2.234023,-0.193792,-0.620762,-0.497495,0.617784,3.066229,-0.870528,4.237797,-1.840606,1.979937,1.048126,0
7979,2.238664,-4.451928,-3.115350,-0.112763,-5.148293,-1.227839,2.607030,1.051266,-1.124752,0.630486,0.719241,-0.239286,2.063776,3.602778,-0.333316,0
3547,-1.136602,3.334867,0.136153,-2.736952,1.059928,-0.454829,1.358385,-1.262984,-0.910792,-0.094602,-0.850578,-0.491673,4.966391,-1.601885,-1.390022,0
4584,-3.037861,2.146766,0.888396,2.789503,5.806704,-2.132121,0.105169,-0.686753,-2.273966,0.606631,-0.345415,1.056435,1.038290,3.742563,-5.362800,0
3947,-0.242303,-1.731569,-1.704046,0.202220,-0.519026,-0.741782,1.579158,-1.430196,2.061192,1.230212,0.227583,0.446696,3.554255,1.822372,-1.342016,0


In [3]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Split positive and negative datasets
positive = train[train[CLASS_COLUMN] == CLASS_VALUE]
print(f"Positive records shape (rows, columns): {positive.shape}")

# Train a nearest neighbor model on the negative dataset
neighbors = NearestNeighbors(n_neighbors=MAX_NEIGHBORS, algorithm="ball_tree")
neighbors.fit(train)

# Locate the nearest neighbors to the positive (minority) set,
# and add to the training set.
nn = neighbors.kneighbors(positive, MAX_NEIGHBORS, return_distance=False)
nn_idx = list(set([item for sublist in nn for item in sublist]))
nearest_neighbors = train.iloc[nn_idx, :]

oversample = pd.concat([positive] * 5)
training_set = pd.concat([oversample, nearest_neighbors]).sample(frac=1)

training_set.head()


Positive records shape (rows, columns): (408, 16)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,Class
2303,0.274222,-3.266096,-1.239380,-2.116911,-5.277208,0.009873,-0.152064,3.232532,-0.502239,-0.110357,0.416285,-2.158894,1.299646,1.514053,-0.492109,1
4008,-2.141986,-0.677490,-1.193989,-3.292175,3.617636,-0.263354,0.291014,-0.956307,0.333120,-0.111491,2.129525,2.926579,2.170853,1.919613,1.150475,0
6809,-0.265028,-4.289244,-0.680609,3.735293,-3.600129,0.400657,-0.499392,2.828997,0.401925,-2.642304,-1.984123,-1.341189,-3.645449,0.056322,-0.035453,1
9073,-1.882966,-1.161521,0.274376,-1.968977,1.851584,-1.284759,0.410516,0.184618,1.199998,-2.685272,-1.070290,0.523300,0.335742,0.496056,1.905491,1
4774,-1.832023,-0.711722,2.798747,-2.534025,2.458227,-2.359642,1.254424,-3.485866,-1.589639,0.316790,1.079321,1.488014,1.667136,-2.201005,1.219179,1


In [4]:
from smart_open import open
import yaml

from gretel_client.projects import create_or_get_unique_project
from gretel_client.helpers import poll

# Create a project and model configuration.
project = create_or_get_unique_project(name="boost-imbalanced-synthetic")

# If you want to use a different config or modify it before creating the model,
# try something like this (yes, we have other stock configs in that repo)
#   from gretel_client.projects.models import read_model_config
#   config = read_model_config("synthetics/default")

# Get a csv to work with, just dump out the training_set.
training_set.to_csv("train.csv", index=False)

# Here we just use a shortcut to specify the default synthetics config.
# Yes, you can use other shortcuts to point at some of the other stock configs.
model = project.create_model_obj(
    model_config="synthetics/default", data_source="train.csv"
)


# Upload the training data.  Train the model.
model.submit_cloud()
poll(model)

recs_to_generate = int(len(dataset.values) * (SYNTHETIC_PERCENT / 100.0))

# Use the model to generate synthetic data.
record_handler = model.create_record_handler_obj(
    params={"num_records": recs_to_generate, "max_invalid": recs_to_generate}
)
record_handler.submit_cloud()

poll(record_handler)

synthetic_df = pd.read_csv(record_handler.get_artifact_link("data"), compression="gzip")
synthetic = synthetic_df[
    synthetic_df[CLASS_COLUMN] == CLASS_VALUE
]  # Keep only positive examples

synthetic.head()


INFO: Starting poller


{
    "uid": "624e44e30d63343893cdac20",
    "guid": "model_27RxHRBgGnYBsRsRTFUj7Yom1BL",
    "model_name": "amusing-kind-baboon",
    "runner_mode": "cloud",
    "user_id": "5ece8962492fbf5bd66089f1",
    "user_guid": "user_26U3XlP53DKYE0zmCWSimFvVFNV",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": null,
    "project_id": "624e44d2a784abf6f363ce95",
    "project_guid": "proj_27RxFIh1tLD0mYAqIZRcb76fgcP",
    "status_history": {
        "created": "2022-04-07T01:56:51.158218Z"
    },
    "last_modified": "2022-04-07T01:56:51.386254Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-east-2.amazonaws.com/gretelai/synthetics@sha256:dadb62d3f5e771b08fca1b17e2503feaf08958ac77c74ae01956d366dc39ae77",
    "model_type": "synthetics",
    "config": {
        "schema_version": "1.0",
        "name": null,
        "models": [
      

INFO: Status is created. Model creation has been queued.
WARN: Got interrupt signal.
INFO: Quitting


SystemExit: 0

/Users/drew/.virtualenvs/g/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
df = pd.concat(
    [
        train.assign(Type="train"),
        test.assign(Type="test"),
        synthetic.assign(Type="synthetic"),
    ]
)
df.reset_index(inplace=True)
df.to_csv("combined-boosted-df.csv")
project.upload_artifact("combined-boosted-df.csv")

# Save to local CSV
synthetic.to_csv("boosted-synthetic.csv", index=False)
project.upload_artifact("boosted-synthetic.csv")

print(f"View this project at: {project.get_console_url()}")


In [ ]:
# Visualize distribution of positive and negative examples in our 
# normal vs. boosted datasets

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


def visualize_distributions(test: pd.DataFrame, train: pd.DataFrame, synthetic: pd.DataFrame):
    """ Plot the distribution of positive (e.g. fraud) vs negative 
        e.g. (non-fraud) examples. 
    """
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
    fig = plt.figure(1, figsize=(12, 9))

    dataframes = {
        "test": test,
        "train": train,
        "boosted": pd.concat([train, synthetic])
    }

    idx = 0
    for name, df in dataframes.items():
        df.Class.value_counts().plot.bar(ax=axes[idx], title=name)
        idx+=1

visualize_distributions(test, train, synthetic)

In [ ]:
## Use PCA to visualize highly dimensional data

# We will label each data class as:
# * Training negative: 0
# * Training positive: 1
# * Synthetic positive: 2 (our synthetic data points used to boost training data)
# * Test positive: 3 (not cheating here, we already trained the classifier)

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


def create_visualization_dataframe(train: pd.DataFrame) -> pd.DataFrame:
    # Build a new visualization dataframe from our training data
    train_vis = train

    # Add in positive synthetic results
    train_vis = pd.merge(train, synthetic, indicator=True, how="outer")
    train_vis.loc[(train_vis._merge == "right_only"), "Class"] = 2
    train_vis = train_vis.drop(columns=["_merge"])

    # Add in positive results from the test set
    train_vis = pd.merge(
        train_vis, test[test["Class"] == 1], indicator=True, how="outer"
    )
    train_vis.loc[
        (train_vis._merge == "right_only") | (train_vis._merge == "both"), "Class"
    ] = 3
    train_vis = train_vis.drop(columns=["_merge"])
    return train_vis


def visualize_pca_2d(train_vis: pd.DataFrame):
    X = train_vis.iloc[:, :-1]
    y = train_vis["Class"]

    fig = plt.figure(1, figsize=(12, 9))
    plt.clf()
    plt.cla()

    pca = PCA(n_components=2)
    x_std = StandardScaler().fit_transform(X)
    projected = pca.fit_transform(x_std)

    labels = ["Train Negative", "Train Positive", "Synthetic Positive", "Test Positive"]
    size_map = {0: 25, 1: 50, 2: 75, 3: 50}
    sizes = [size_map[x] for x in y]

    scatter = plt.scatter(
        projected[:, 0], projected[:, 1], c=y, s=sizes, cmap=plt.cm.plasma, alpha=0.8
    )
    plt.title = f"PCA plot of {n_features}-dimension classification dataset"
    plt.legend(handles=scatter.legend_elements()[0], labels=labels)
    plt.show()


# Visualize PCA distribution in 2D
train_vis = create_visualization_dataframe(train)
visualize_pca_2d(train_vis)


In [ ]:
# Plot PCA scatter in 3 dimensions
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn import datasets


def visualize_pca_3d(train_vis: pd.DataFrame):
    X = train_vis.iloc[:, :-1]
    y = train_vis["Class"]

    np.random.seed(5)

    fig = plt.figure(1, figsize=(12, 9))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
    plt.cla()
    pca = decomposition.PCA(n_components=3)
    labels = ["Train Negative", "Train Positive", "Synthetic Positive", "Test Positive"]
    size_map = {0: 25, 1: 50, 2: 75, 3: 50}
    sizes = [size_map[x] for x in y]

    pca.fit(X)
    X = pca.transform(X)

    scatter = ax.scatter(
        X[:, 0], X[:, 1], X[:, 2], c=y, s=sizes, cmap=plt.cm.plasma, alpha=1.0
    )

    plt.legend(handles=scatter.legend_elements()[0], labels=labels)
    plt.show()


# Visualize PCA distribution in 3D
visualize_pca_3d(train_vis)


In [ ]:
# Train an XGBoost model and compare accuracies on the original (normal)
# vs. augmented training data (train + synthetic) datasets.

from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier


def train_classifier(name: str, train: pd.DataFrame, test: pd.DataFrame):
    """Train our predictor with XGBoost"""

    # Encode labels and categorical variables before training prediction model
    X_train = train.iloc[:, :-1]
    y_train = train["Class"]
    X_test = test.iloc[:, :-1]
    y_test = test["Class"]

    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    np.set_printoptions(precision=2)
    print("%s : XGBoost Model prediction accuracy: %.2f%%" % (name, accuracy * 100.0))
    return model, y_pred


# Train models on normal and augmented data
model_normal, y_pred = train_classifier("normal", train, test)
model_boosted, y_pred = train_classifier("boosted", pd.concat([train, synthetic]), test)


In [ ]:
# A confusion matrix gives better insight into per-class performance
# than overall model accuracy.

# As a thought experiment, consider creating a model to predict whether
# an account will submit an insurance claim. Our goal is to maximize
# accuracy at predicting the minority (positive) set, above those who
# will not submit a claim. Try to maximize the diagonal (TP) elements of the
# confusion matrix, particularly the bottom right.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix


def print_confusion_matrix(name: str, model: pd.DataFrame, test: pd.DataFrame):
    """Plot normalized and non-normalized confusion matrices"""
    print("")
    print("")
    print(f"Plotting confusion matrices for: {name} model")
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    fig = plt.figure(1, figsize=(12, 9))
    X_test = test.iloc[:, :-1]
    y_test = test["Class"]

    titles_options = [
        (f"{name} : Confusion matrix, without normalization", None),
        (f"{name} : Normalized confusion matrix", "true"),
    ]

    idx = 0
    for title, normalize in titles_options:
        disp = plot_confusion_matrix(
            model,
            X_test,
            y_test,
            display_labels=["Negative", "Positive"],
            cmap=plt.cm.Blues,
            normalize=normalize,
            ax=axes[idx],
        )
        disp.ax_.set_title(title)
        idx += 1

    plt.show()


print_confusion_matrix("normal", model_normal, test)
print_confusion_matrix("boosted", model_boosted, test)
